# U.S. Opiate Prescriptions

Accidental death by fatal drug overdose is a rising trend in the United States. What can you do to help?

This dataset contains summaries of prescription records for 250 common opioid and non-opioid drugs written by 25,000 unique licensed medical professionals in 2014 in the United States for citizens covered under Class D Medicare as well as some metadata about the doctors themselves. This is a small subset of data that was sourced from cms.gov. The full dataset contains almost 24 million prescription instances in long format. I have cleaned and compiled this data here in a format with 1 row per prescriber and limited the approximately 1 million total unique prescribers down to 25,000 to keep it manageable. If you are interested in more data, you can get the script I used to assemble the dataset here and run it yourself. The main data is in prescriber-info.csv. There is also opioids.csv that contains the names of all opioid drugs included in the data and overdoses.csv that contains information on opioid related drug overdose fatalities.

### A key for reading the dataset

NPI – unique National Provider Identifier number

Gender - (M/F)

State - US State by abbreviation

Credentials - set of initials indicative of medical degree

Specialty - description of type of medicinal practice

Opioid.Prescriber - a boolean label indicating whether or not that individual prescribed opiate drugs more than 10 times in the year

In [58]:
import pandas as pd
import numpy as np

In [5]:
missing_values = ['n/a', 'na', '--']
opioids = pd.read_csv('opioids.csv', na_values = missing_values)
overdoses = pd.read_csv('overdoses.csv', na_values = missing_values, thousands= ',')
prescriber = pd.read_csv('prescriber-info.csv', na_values = missing_values, thousands= ',')

# Exploratory Analysis

In [96]:
list(opioids['Drug Name'])

['ABSTRAL',
 'ACETAMINOPHEN-CODEINE',
 'ACTIQ',
 'ASCOMP WITH CODEINE',
 'ASPIRIN-CAFFEINE-DIHYDROCODEIN',
 'AVINZA',
 'BELLADONNA-OPIUM',
 'BUPRENORPHINE HCL',
 'BUTALB-ACETAMINOPH-CAFF-CODEIN',
 'BUTALB-CAFF-ACETAMINOPH-CODEIN',
 'BUTALBITAL COMPOUND-CODEINE',
 'BUTORPHANOL TARTRATE',
 'BUTRANS',
 'CAPITAL W-CODEINE',
 'CARISOPRODOL COMPOUND-CODEINE',
 'CARISOPRODOL-ASPIRIN-CODEINE',
 'CODEINE SULFATE',
 'CO-GESIC',
 'CONZIP',
 'DEMEROL',
 'DEMEROL',
 'DILAUDID',
 'DILAUDID',
 'DILAUDID-HP',
 'DISKETS',
 'DOLOPHINE HCL',
 'DURAGESIC',
 'DURAMORPH',
 'ENDOCET',
 'ENDODAN',
 'EXALGO',
 'FENTANYL',
 'FENTANYL CITRATE',
 'FENTORA',
 'FIORICET WITH CODEINE',
 'FIORINAL WITH CODEINE #3',
 'HYCET',
 'HYDROCODONE-ACETAMINOPHEN',
 'HYDROCODONE-IBUPROFEN',
 'HYDROMORPHONE ER',
 'HYDROMORPHONE HCL',
 'HYDROMORPHONE HCL',
 'IBUDONE',
 'INFUMORPH',
 'KADIAN',
 'LAZANDA',
 'LEVORPHANOL TARTRATE',
 'LORCET',
 'LORCET 10-650',
 'LORCET HD',
 'LORCET PLUS',
 'LORTAB',
 'MAGNACET',
 'MEPERIDINE HCL',


In [87]:
overdoses

,State,Population,Deaths,Abbrev,DeathRatio
0,Alabama,4833722,723,AL,0.000150
1,Alaska,735132,124,AK,0.000169
2,Arizona,6626624,1211,AZ,0.000183
3,Arkansas,2959373,356,AR,0.000120
4,California,38332521,4521,CA,0.000118
5,Colorado,5268367,899,CO,0.000171
6,Connecticut,3596080,623,CT,0.000173
7,Delaware,925749,189,DE,0.000204
8,Florida,19552860,2634,FL,0.000135
9,Georgia,9992167,1206,GA,0.000121


In [80]:
### Prescriber Data Description

In [88]:
prescriber

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,New
0,1710982582,M,TX,Dentist-DDS-DDS,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Dentist-DDS-DDS
1,1245278100,F,AL,General Surgery-MD-MD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,35,1,General Surgery-MD-MD
2,1427182161,F,NY,General Practice-MD-MD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25,0,General Practice-MD-MD
3,1669567541,M,AZ,Internal Medicine-MD-MD,0,43,0,0,0,21,...,0,0,0,0,0,0,0,0,1,Internal Medicine-MD-MD
4,1679650949,M,NV,Hematology/Oncology-MD-MD,0,0,0,0,0,0,...,0,0,0,17,28,0,0,0,1,Hematology/Oncology-MD-MD
5,1548580897,M,PA,General Surgery-DO-DO,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,General Surgery-DO-DO
6,1437192002,M,NH,Family Practice-MD-MD,0,0,0,25,0,0,...,0,0,0,80,0,16,0,38,1,Family Practice-MD-MD
7,1407113988,F,PA,"Nurse Practitioner-RN, MSN, ANP-BC-RN, MSN, AN...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Nurse Practitioner-RN, MSN, ANP-BC-RN, MSN, AN..."
8,1023260569,M,TX,Optometry-OD-OD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Optometry-OD-OD
9,1821106832,F,WI,Internal Medicine-MD-MD,0,0,0,11,0,15,...,0,0,0,56,0,20,0,0,1,Internal Medicine-MD-MD


In [30]:
# Remove (.) from Credentials column
prescriber['Credentials'] = prescriber['Credentials'].str.replace('.', '')

In [7]:
#List the column names to find any common drug names
list(prescriber.columns)

['NPI',
 'Gender',
 'State',
 'Credentials',
 'Specialty',
 'ABILIFY',
 'ACETAMINOPHEN.CODEINE',
 'ACYCLOVIR',
 'ADVAIR.DISKUS',
 'AGGRENOX',
 'ALENDRONATE.SODIUM',
 'ALLOPURINOL',
 'ALPRAZOLAM',
 'AMIODARONE.HCL',
 'AMITRIPTYLINE.HCL',
 'AMLODIPINE.BESYLATE',
 'AMLODIPINE.BESYLATE.BENAZEPRIL',
 'AMOXICILLIN',
 'AMOX.TR.POTASSIUM.CLAVULANATE',
 'AMPHETAMINE.SALT.COMBO',
 'ATENOLOL',
 'ATORVASTATIN.CALCIUM',
 'AVODART',
 'AZITHROMYCIN',
 'BACLOFEN',
 'BD.ULTRA.FINE.PEN.NEEDLE',
 'BENAZEPRIL.HCL',
 'BENICAR',
 'BENICAR.HCT',
 'BENZTROPINE.MESYLATE',
 'BISOPROLOL.HYDROCHLOROTHIAZIDE',
 'BRIMONIDINE.TARTRATE',
 'BUMETANIDE',
 'BUPROPION.HCL.SR',
 'BUPROPION.XL',
 'BUSPIRONE.HCL',
 'BYSTOLIC',
 'CARBAMAZEPINE',
 'CARBIDOPA.LEVODOPA',
 'CARISOPRODOL',
 'CARTIA.XT',
 'CARVEDILOL',
 'CEFUROXIME',
 'CELEBREX',
 'CEPHALEXIN',
 'CHLORHEXIDINE.GLUCONATE',
 'CHLORTHALIDONE',
 'CILOSTAZOL',
 'CIPROFLOXACIN.HCL',
 'CITALOPRAM.HBR',
 'CLINDAMYCIN.HCL',
 'CLOBETASOL.PROPIONATE',
 'CLONAZEPAM',
 'CLONID

In [8]:
# Join common drugs together to clean up the dataframe
prescriber['AMLODIPINE.BESYLATE'] = (prescriber['AMLODIPINE.BESYLATE'] +
  prescriber['AMLODIPINE.BESYLATE.BENAZEPRIL'])

prescriber['BENICAR'] = (prescriber['BENICAR'] + prescriber['BENICAR.HCT'])

prescriber['BUPROPION'] = (prescriber['BUPROPION.HCL.SR'] + prescriber['BUPROPION.XL'])

prescriber['DILTIAZEM'] = (prescriber['DILTIAZEM.24HR.CD'] 
                           + prescriber['DILTIAZEM.24HR.ER'] + (prescriber['DILTIAZEM.24HR.ER'] 
                           + prescriber['DILTIAZEM.ER']) + prescriber['DILTIAZEM.HCL'])

prescriber['DIVALPROEX.SODIUM'] = (prescriber['DIVALPROEX.SODIUM'] 
                           + prescriber['DIVALPROEX.SODIUM.ER'])

prescriber['DIGOX'] = (prescriber['DIGOX'] + prescriber['DIGOXIN'])
prescriber['GLIPIZIDE'] = (prescriber['GLIPIZIDE'] 
                           + prescriber['GLIPIZIDE.ER'] + prescriber['GLIPIZIDE.XL'])

prescriber['KLOR.CON'] = (prescriber['KLOR.CON.10'] 
                           + prescriber['KLOR.CON.M10'] + prescriber['KLOR.CON.M20'])

prescriber['LANTUS'] = (prescriber['LANTUS'] + prescriber['LANTUS.SOLOSTAR'])

prescriber['LEVEMIR'] = (prescriber['LEVEMIR'] + prescriber['LEVEMIR.FLEXPEN'])


prescriber['LISINOPRIL'] = (prescriber['LISINOPRIL'] + prescriber['LISINOPRIL.HYDROCHLOROTHIAZIDE'])

prescriber['LOSARTAN.HYDROCHLOROTHIAZIDE'] = (prescriber['LOSARTAN.HYDROCHLOROTHIAZIDE'] 
                                               + prescriber['LOSARTAN.POTASSIUM'])

prescriber['METFORMIN.HCL'] = (prescriber['METFORMIN.HCL'] + prescriber['METFORMIN.HCL.ER'])

prescriber['METOPROLOL'] = (prescriber['METOPROLOL.SUCCINATE'] + prescriber['METOPROLOL.TARTRATE'])

prescriber['MORPHINE.SULFATE'] = (prescriber['MORPHINE.SULFATE'] + prescriber['MORPHINE.SULFATE.ER'])

prescriber['NAMENDA'] = (prescriber['NAMENDA'] + prescriber['NAMENDA.XR'])

prescriber['NIFEDIPINER'] = (prescriber['NIFEDIPINE.ER'] + prescriber['NIFEDIPINE.ER'])

prescriber['NOVOLOG'] = (prescriber['NOVOLOG'] + prescriber['NOVOLOG.FLEXPEN'])

prescriber['ONDANSETRON'] = (prescriber['ONDANSETRON.HCL'] + prescriber['ONDANSETRON.ODT'])

prescriber['OXYBUTYNIN.CHLORIDE'] = (prescriber['OXYBUTYNIN.CHLORIDE'] + prescriber['OXYBUTYNIN.CHLORIDE.ER'])

prescriber['OXYCODONE'] = (prescriber['OXYCODONE.ACETAMINOPHEN'] + prescriber['OXYCODONE.HCL'])

prescriber['PROPRANOLOL.HCL'] = (prescriber['PROPRANOLOL.HCL'] + prescriber['PROPRANOLOL.HCL.ER'])

prescriber['VALSARTAN'] = (prescriber['VALSARTAN'] + prescriber['VALSARTAN.HYDROCHLOROTHIAZIDE'])

prescriber['VENLAFAXINE'] = (prescriber['VENLAFAXINE.HCL'] + prescriber['VENLAFAXINE.HCL.ER'])

In [9]:
#Create variable to collect the duplicated drug name columns needed to be dropped
dup_cols = ['AMLODIPINE.BESYLATE.BENAZEPRIL', 'BENICAR.HCT','BUPROPION.HCL.SR', 'BUPROPION.XL','DIGOXIN','DILTIAZEM.24HR.CD','DILTIAZEM.24HR.ER','DILTIAZEM.ER','DILTIAZEM.HCL','DIVALPROEX.SODIUM.ER',
'GLIPIZIDE.ER', 'GLIPIZIDE.XL', 'KLOR.CON.10','KLOR.CON.M10','KLOR.CON.M20','LANTUS.SOLOSTAR','LEVEMIR.FLEXPEN', 'LISINOPRIL.HYDROCHLOROTHIAZIDE',
'LOSARTAN.POTASSIUM', 'METFORMIN.HCL.ER', 'METOPROLOL.SUCCINATE', 'METOPROLOL.TARTRATE', 'MORPHINE.SULFATE.ER','NAMENDA.XR','NOVOLOG.FLEXPEN',
'ONDANSETRON.HCL', 'ONDANSETRON.ODT','OXYBUTYNIN.CHLORIDE.ER', 'OXYCODONE.ACETAMINOPHEN', 'OXYCODONE.HCL','PROPRANOLOL.HCL.ER',
'VALSARTAN.HYDROCHLOROTHIAZIDE','VENLAFAXINE.HCL', 'VENLAFAXINE.HCL.ER']

In [10]:
#Drop duplicate columns
prescriber = prescriber.drop(dup_cols, axis=1)

In [42]:
#Drop duplicate Credential column
prescriber = prescriber.drop(['Credentials'], axis=1)

In [12]:
#Reindex dataframe in alphabetical order
prescriber = prescriber.reindex(sorted(prescriber.columns), axis=1)

In [13]:
# Now that the dataframe is alphabetized it's time to return the dataframe back to original order

# Move Opioid.Prescriber to the end of the dataframe
cols_to_move = ['Opioid.Prescriber']
new_cols = np.hstack((prescriber.columns.difference(cols_to_move), cols_to_move))
prescriber = prescriber.loc[:, new_cols]

In [14]:
# Move NPI,Gender,State,Credentials, Specialty back to the front of the dataframe
col_to_move = ['NPI','Gender','State','Specialty']
prescriber = prescriber[col_to_move + [col for col in prescriber.columns if col not in col_to_move ]]
prescriber.head()

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,Dentist,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,General Surgery,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,General Practice,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,Internal Medicine,0,43,0,0,0,21,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,Hematology/Oncology,0,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [15]:
# Identify rows with NaN in Specialty column
prescriber.loc[prescriber.isnull().any(axis=1)]

# Remove rows with Nan in Specialty column
prescriber = prescriber.dropna()

In [90]:
prescriber[(prescriber.ABILIFY >=1)]

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,New
36,1902018518,F,NV,Nurse Practitioner-APRN-APRN,294,0,0,0,0,0,...,0,0,0,0,0,0,95,192,0,Nurse Practitioner-APRN-APRN
39,1285675637,M,NE,Family Practice-MD-MD,11,0,0,119,0,14,...,0,0,0,176,13,0,0,34,1,Family Practice-MD-MD
74,1912114752,M,OH,Psychiatry-MD-MD,25,0,0,0,0,0,...,0,0,0,0,0,0,0,19,1,Psychiatry-MD-MD
82,1871682567,F,TN,Nurse Practitioner-NP-NP,30,0,0,22,0,22,...,0,0,0,31,0,0,0,28,1,Nurse Practitioner-NP-NP
107,1093901357,M,OH,Internal Medicine-MD-MD,12,0,0,19,0,17,...,0,0,0,0,0,0,0,172,1,Internal Medicine-MD-MD
138,1437209079,F,NY,Psychiatry & Neurology-MD-MD,48,0,0,0,0,0,...,0,0,0,0,0,0,0,33,0,Psychiatry & Neurology-MD-MD
153,1508931650,M,IN,Family Practice-MD-MD,16,23,0,46,0,32,...,0,21,0,29,38,0,0,67,1,Family Practice-MD-MD
200,1104835941,F,NM,"Certified Clinical Nurse Specialist-PMH, CNP/C...",85,0,0,0,0,0,...,0,0,0,0,0,0,93,72,0,"Certified Clinical Nurse Specialist-PMH, CNP/C..."
211,1649208638,F,CA,Neuropsychiatry-MD-MD,45,0,0,0,0,0,...,0,0,0,0,0,0,36,25,1,Neuropsychiatry-MD-MD
227,1306886064,M,KY,Diagnostic Radiology-MD-MD,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Diagnostic Radiology-MD-MD


In [20]:
#Create column in Overdoses to show death ratio
overdoses['DeathRatio'] = overdoses['Deaths']/overdoses['Population']

In [21]:
overdoses.groupby(['Deaths'])['DeathRatio'].mean()

Deaths
43      0.000059
63      0.000075
83      0.000132
109     0.000187
124     0.000169
125     0.000095
157     0.000112
189     0.000204
212     0.000132
216     0.000163
247     0.000235
264     0.000085
332     0.000115
334     0.000252
336     0.000112
356     0.000120
517     0.000095
522     0.000133
545     0.000195
547     0.000262
603     0.000208
623     0.000173
627     0.000338
701     0.000147
723     0.000150
777     0.000185
853     0.000149
899     0.000171
979     0.000140
980     0.000119
1067    0.000177
1070    0.000180
1077    0.000245
1172    0.000178
1206    0.000121
1211    0.000183
1253    0.000141
1269    0.000195
1289    0.000193
1358    0.000138
1705    0.000132
1762    0.000178
2300    0.000117
2601    0.000098
2634    0.000135
2732    0.000214
2744    0.000237
4521    0.000118
Name: DeathRatio, dtype: float64

In [43]:
prescriber.head()

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,Dentist-DDS-DDS,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,General Surgery-MD-MD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,General Practice-MD-MD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,Internal Medicine-MD-MD,0,43,0,0,0,21,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,Hematology/Oncology-MD-MD,0,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [45]:
# Find any NaN listings in the Specialty column
prescriber['Specialty'].isnull().sum()

763

In [46]:
# Remove any Nan listings in the Specialty column
prescriber = prescriber.dropna()

In [55]:
b = prescriber

In [ ]:
# drop non US States: PR, AA, GU, AE, ZZ
#df = df[(df.State != 'GU') & (df.State != 'AA')]

In [ ]:
# Rename DC to VA
#df['State'] = df['State'].str.replace('DC', 'VA')

In [116]:
list(opioids['Drug Name'])

['ABSTRAL',
 'ACETAMINOPHEN-CODEINE',
 'ACTIQ',
 'ASCOMP WITH CODEINE',
 'ASPIRIN-CAFFEINE-DIHYDROCODEIN',
 'AVINZA',
 'BELLADONNA-OPIUM',
 'BUPRENORPHINE HCL',
 'BUTALB-ACETAMINOPH-CAFF-CODEIN',
 'BUTALB-CAFF-ACETAMINOPH-CODEIN',
 'BUTALBITAL COMPOUND-CODEINE',
 'BUTORPHANOL TARTRATE',
 'BUTRANS',
 'CAPITAL W-CODEINE',
 'CARISOPRODOL COMPOUND-CODEINE',
 'CARISOPRODOL-ASPIRIN-CODEINE',
 'CODEINE SULFATE',
 'CO-GESIC',
 'CONZIP',
 'DEMEROL',
 'DEMEROL',
 'DILAUDID',
 'DILAUDID',
 'DILAUDID-HP',
 'DISKETS',
 'DOLOPHINE HCL',
 'DURAGESIC',
 'DURAMORPH',
 'ENDOCET',
 'ENDODAN',
 'EXALGO',
 'FENTANYL',
 'FENTANYL CITRATE',
 'FENTORA',
 'FIORICET WITH CODEINE',
 'FIORINAL WITH CODEINE #3',
 'HYCET',
 'HYDROCODONE-ACETAMINOPHEN',
 'HYDROCODONE-IBUPROFEN',
 'HYDROMORPHONE ER',
 'HYDROMORPHONE HCL',
 'HYDROMORPHONE HCL',
 'IBUDONE',
 'INFUMORPH',
 'KADIAN',
 'LAZANDA',
 'LEVORPHANOL TARTRATE',
 'LORCET',
 'LORCET 10-650',
 'LORCET HD',
 'LORCET PLUS',
 'LORTAB',
 'MAGNACET',
 'MEPERIDINE HCL',


In [123]:
opioids['Drug Name'].mode()

0              DEMEROL
1             DILAUDID
2    HYDROMORPHONE HCL
3       MEPERIDINE HCL
4     MORPHINE SULFATE
dtype: object

In [100]:
list(opioids['Drug Name'])

['ABSTRAL',
 'ACETAMINOPHEN-CODEINE',
 'ACTIQ',
 'ASCOMP WITH CODEINE',
 'ASPIRIN-CAFFEINE-DIHYDROCODEIN',
 'AVINZA',
 'BELLADONNA-OPIUM',
 'BUPRENORPHINE HCL',
 'BUTALB-ACETAMINOPH-CAFF-CODEIN',
 'BUTALB-CAFF-ACETAMINOPH-CODEIN',
 'BUTALBITAL COMPOUND-CODEINE',
 'BUTORPHANOL TARTRATE',
 'BUTRANS',
 'CAPITAL W-CODEINE',
 'CARISOPRODOL COMPOUND-CODEINE',
 'CARISOPRODOL-ASPIRIN-CODEINE',
 'CODEINE SULFATE',
 'CO-GESIC',
 'CONZIP',
 'DEMEROL',
 'DEMEROL',
 'DILAUDID',
 'DILAUDID',
 'DILAUDID-HP',
 'DISKETS',
 'DOLOPHINE HCL',
 'DURAGESIC',
 'DURAMORPH',
 'ENDOCET',
 'ENDODAN',
 'EXALGO',
 'FENTANYL',
 'FENTANYL CITRATE',
 'FENTORA',
 'FIORICET WITH CODEINE',
 'FIORINAL WITH CODEINE #3',
 'HYCET',
 'HYDROCODONE-ACETAMINOPHEN',
 'HYDROCODONE-IBUPROFEN',
 'HYDROMORPHONE ER',
 'HYDROMORPHONE HCL',
 'HYDROMORPHONE HCL',
 'IBUDONE',
 'INFUMORPH',
 'KADIAN',
 'LAZANDA',
 'LEVORPHANOL TARTRATE',
 'LORCET',
 'LORCET 10-650',
 'LORCET HD',
 'LORCET PLUS',
 'LORTAB',
 'MAGNACET',
 'MEPERIDINE HCL',


In [118]:
opioids['Generic Name'].value_counts().idxmax()

'HYDROCODONE/ACETAMINOPHEN'

In [ ]:
prescriber['Credentials'] = prescriber['Credentials'].str.replace('.', '')

In [99]:
ops = list(opioids['Drug Name'])
prescribed_ops = list(set(ops) & set(prescriber.columns))

for i,drug in enumerate(prescribed_ops):
    print (i+1,drug)

1 FENTANYL
2 OXYCONTIN


In [103]:
df = pd.DataFrame(np.array([[1, 1], [2, 10], [3, 100], [4, 100]]),columns=['a', 'b'])
df.quantile(.1)

,a,b
0,1,1
1,2,10
2,3,100
3,4,100


In [105]:
df.quantile(.1, .25, .5, .75)

ValueError: No axis named 0.25 for object type <class 'type'>

In [113]:
z = prescriber
z1 = z[(z.State != 'GU') & (z.State != 'AA') & (z.State != 'PR') & (z.State != 'AE') & (z.State != 'ZZ') & (z.State != 'VI')]
z1['State'].value_counts()

CA    2487
NY    1877
FL    1530
TX    1462
PA    1179
IL     965
OH     949
MI     852
NC     748
MA     698
NJ     625
GA     602
WA     556
VA     553
TN     539
IN     517
AZ     495
WI     485
MD     476
MO     473
MN     414
CO     379
SC     379
CT     374
KY     361
LA     341
AL     336
OR     336
OK     275
IA     223
AR     208
KS     202
WV     195
MS     192
UT     160
NM     159
NV     154
ME     145
NE     137
ID     131
NH     118
RI     114
DE      87
HI      85
SD      82
MT      77
DC      73
ND      65
VT      63
AK      39
WY      38
Name: State, dtype: int64

In [124]:
z2 = z.drop(z.index[z['State'] == ('GU', 'PR', 'AA', 'AE', 'ZZ', 'VI')])
z2['State'].value_counts()

CA    2487
NY    1877
FL    1530
TX    1462
PA    1179
IL     965
OH     949
MI     852
NC     748
MA     698
NJ     625
GA     602
WA     556
VA     553
TN     539
IN     517
AZ     495
WI     485
MD     476
MO     473
MN     414
CO     379
SC     379
CT     374
KY     361
LA     341
OR     336
AL     336
OK     275
IA     223
PR     217
AR     208
KS     202
WV     195
MS     192
UT     160
NM     159
NV     154
ME     145
NE     137
ID     131
NH     118
RI     114
DE      87
HI      85
SD      82
MT      77
DC      73
ND      65
VT      63
AK      39
WY      38
VI       3
GU       2
ZZ       2
AE       2
AA       1
Name: State, dtype: int64